In [18]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

import os
os.chdir('/home/cat/code/manifolds/')

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import scipy
import numpy as np
import pandas as pd

from calcium import calcium
from wheel import wheel
from visualize import visualize
from tqdm import trange, tqdm

from scipy.io import loadmat

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.spatial import ConvexHull, convex_hull_plot_2d

import sklearn

# 
np.set_printoptions(suppress=True)


#from utils import load_UMAP, load_binarized_traces, find_ensemble_order, load_data, HMM, get_footprint_contour, load_footprints, computing_ensemble_loadings



Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:

def plot_ensembles_side_by_side(states, c):
    #
    
    # find which cells are good 
    idx2 = np.where(c.iscell[:,0])[0]


    fig=plt.figure(figsize=(20,20))
    for k in trange(len(states)):
        #ax=plt.subplot(10,8,k+1)
        ax=plt.subplot(5,5,k+1)
        idx_state = states[k][0].squeeze()

        # find which states
        idx_good = []
        for id_ in idx_state:
            if id_ in idx2:
                idx_good.append(id_)

        #print (idx_good)

        #
        final_ids = []
        for p in range(len(idx_good)):
            temp = idx_good[p]
            idx4 = np.where(idx2==temp)[0]
            final_ids.append(idx4)
            p#rint (temp, idx4)
        final_ids = np.hstack(final_ids)

        # 
        for id2 in final_ids:
            contour = c.get_footprint_contour(id2)
            plt.plot(contour[:,1],
                    contour[:,0],
                    c='red')

        # plot all ensembles:
        for id2 in range(len(c.footprints_bin)):
            contour = c.get_footprint_contour(id2)
            plt.plot(contour[:,1],
                    contour[:,0],
                    c='black',
                    alpha=.15)

        #
        plt.xlim(0,512)
        plt.ylim(0,512)
        plt.xticks([])
        plt.yticks([])
        plt.ylabel(str(k) + ", "+str(len(final_ids)))
        
def plot_ensembles_together(states, c):
    
    # enumerate post-cleaned cells
    all_cells = np.arange(c.F.shape[0])

    # find which cells are good 
    idx2 = np.where(c.iscell[:,0])[0]
    print (idx2.shape)



    # 
    cmap = matplotlib.cm.get_cmap('gist_ncar', len(states))
    
    #
    fig=plt.figure(figsize=(20,20))
    for k in trange(len(states)):
        #ax=plt.subplot(10,8,k+1)
        #ax=plt.subplot(5,5,k+1)
        idx_state = states[k][0].squeeze()

        # find which states
        idx_good = []
        for id_ in idx_state:
            if id_ in idx2:
                idx_good.append(id_)

        #print (idx_good)

        #
        final_ids = []
        for p in range(len(idx_good)):
            temp = idx_good[p]
            idx4 = np.where(idx2==temp)[0]
            final_ids.append(idx4)
            p#rint (temp, idx4)
        final_ids = np.hstack(final_ids)

        # 
        shiftx = np.random.randint(-3,3)
        shifty = np.random.randint(-3,3)
        if shiftx==0:
            shiftx=1
        if shifty==0:
            shifty=1
        
        #scale=2
        # 
        for id2 in final_ids:
            contour = c.get_footprint_contour(id2)
            
            if True:
                plt.plot(
                    contour[:,1]+shiftx,
                    contour[:,0]+shifty,
                    c=cmap(k),
                    linewidth=2)
            else:
                plt.scatter(
                       np.mean(contour[:,1])+shiftx,
                np.mean(contour[:,0])+shifty,
                c=cmap(k),
                s=50)
                
                
    # plot all ensembles:
    for id2 in range(len(c.footprints_bin)):
        contour = c.get_footprint_contour(id2)
        plt.plot(contour[:,1],
                contour[:,0],
                c='black',
                alpha=.15)

    #
    plt.xlim(0,512)
    plt.ylim(0,512)
    plt.xticks([])
    plt.yticks([])
    plt.ylabel(str(k) + ", "+str(len(final_ids)))


In [28]:
#############################################################################
##### DISTRIBUTIONS OF # NEURONS IN EACH ENSEMBLE VS. DEVELOPMENT ###########
#############################################################################
root_dir = '/media/cat/4TB/donato/'
animal_id = 'DON-006084'

df = pd.read_excel('/media/cat/4TB/donato/CA3 Wheel Animals Database.xlsx')

# 
root_dir = '/media/cat/4TB/donato/'
animal_id = 'DON-006084'
# animal_id = 'DON-006087'
# animal_id = 'DON-003343'
# animal_id = 'DON-003165'
#session = '20210520'

binarization_method='upphase'

idx = np.where(df['Mouse_id']==animal_id)[0].squeeze()
sessions = df.iloc[idx]['Session_ids'].split(',')
print ("sessions: ", sessions)

#
fig=plt.figure()
cmap = plt.get_cmap("viridis", 10)
ctr=0
for session in sessions:
    session = session.replace(' ','')
    
    F = np.load(os.path.join(root_dir,
                              animal_id,
                              session,
                             'suite2p',
                             'plane0',
                             'ensembles',
                            'F_upphase_binned_sum.npy'))
    print ("# neurons: ", F.shape[1])
    # 
    ensembles = scipy.io.loadmat(os.path.join(root_dir,
                                          animal_id,
                                          session,
                                         'suite2p',
                                         'plane0',
                                         'ensembles',
                                         'Upphase_sum_binned_ICA_ASSEMBLIES.mat'))
    states = ensembles['cs_assemblies'].squeeze()
    print (session,F.shape, states.shape)
    
    lens = []
    norm=False
    for k in range(states.shape[0]):
        #print (states[k].squeeze().shape)
        n_neurons = states[k].squeeze().shape[0]
        if norm:
            n_neurons = n_neurons/F.shape[1]
        
        
        lens.append(n_neurons)
        
    if norm:
        y = np.histogram(lens, bins=np.arange(-0.0,0.15,0.01))
    else:
        y = np.histogram(lens, bins=np.arange(0.0,100,5))
        
    plt.plot(y[1][:-1],
             y[0],#/np.max(y[0]),
            color=cmap(ctr))
    plt.ylabel('# of ensembles')
    if norm:
        plt.xlabel("proportion of neurons in the ensemble")
    else:
        plt.xlabel("# of neurons in the ensemble")
    ctr+=1
    break
plt.suptitle(animal_id)
plt.show()

sessions:  ['20210519', ' 20210520', ' 20210521', ' 20210522', ' 20210523', ' 20210524', ' 20210525', ' 20210526', ' 20210527', ' 20210528']
# neurons:  531
20210519 (3716, 531) (25,)


In [29]:
################################################################################################
##### DISTRIBUTIONS OF MEDIAN INTER-NEURON DISTANCE IN EACH ENSEMBLE VS. DEVELOPMENT ###########
################################################################################################
root_dir = '/media/cat/4TB/donato/'
animal_id = 'DON-006084'

df = pd.read_excel('/media/cat/4TB/donato/CA3 Wheel Animals Database.xlsx')


# 
root_dir = '/media/cat/4TB/donato/'
#animal_id = 'DON-006084'
#animal_id = 'DON-006087'
# animal_id = 'DON-003343'
animal_id = 'DON-003165'
#session = '20210520'

binarization_method='upphase'

idx = np.where(df['Mouse_id']==animal_id)[0].squeeze()
sessions = df.iloc[idx]['Session_ids'].split(',')
print ("sessions: ", sessions)
#
fig=plt.figure()
cmap = plt.get_cmap("viridis", 10)
ctr=0
for session in sessions:
    session = session.replace(' ','')
    
    F = np.load(os.path.join(root_dir,
                              animal_id,
                              session,
                             'suite2p',
                             'plane0',
                             'ensembles',
                            'F_upphase_binned_sum.npy'))
    print ("# neurons: ", F.shape[1])
    # 
    ensembles = scipy.io.loadmat(os.path.join(root_dir,
                                          animal_id,
                                          session,
                                         'suite2p',
                                         'plane0',
                                         'ensembles',
                                         'Upphase_sum_binned_ICA_ASSEMBLIES.mat'))
    states = ensembles['cs_assemblies'].squeeze()
    print (session,F.shape, states.shape)

    
    c = calcium.Calcium()
    c.root_dir = root_dir
    c.animal_id = animal_id
    c.session = session
    c.load_suite2p()
    
    c.load_footprints()
    

    median_dists = []
    for k in range(states.shape[0]):
        #print (states[k].squeeze().shape)
        ids = states[k].squeeze()

        #print ("ids: ", ids)
        contours_centres = np.zeros((ids.shape[0],2))
        for ctr2, i in enumerate(ids):
            temp = c.get_footprint_contour(i-1)
            tempxy = np.median(temp, axis=0)
            contours_centres[ctr2] = tempxy
        
        dists = []
        for z in range(contours_centres.shape[0]):
            for z2 in range(z+1, contours_centres.shape[0],1):
                temp = sklearn.metrics.pairwise.euclidean_distances(contours_centres[z].reshape(-1,1),
                                                                   contours_centres[z2].reshape(-1,1))
                dists.append(temp)
                
        median_dists.append(np.median(dists))
 
    # 
    y = np.histogram(median_dists, bins=np.arange(0.0,200,10))
        
    plt.plot(y[1][:-1],
             y[0],#/np.max(y[0]),
             color=cmap(ctr))
    plt.ylabel('# of ensembles')
    plt.xlabel("median distance between pairs of neurons")
    ctr+=1
    break
plt.suptitle(animal_id)
plt.show()

sessions:  ['DON-003165_20210211', ' DON-003165_20210213', ' DON-003165_20210215', ' DON-003165_20210216', ' DON-003165_20210217', ' DON-003165_20210218', ' DON-003165_20210219', ' DON-003165_20210220', ' DON-003165_20210222', ' DON-003165_20210223']
# neurons:  657
DON-003165_20210211 (3716, 657) (5,)




In [ ]:
c.get_footprint_contour()